In [ ]:
import pandas as pd
import numpy as np
import os
os.chdir('../')
from kernel import utils, data_processing, modelling

# 1. Load Training Data

In [ ]:
df = pd.read_csv('./data/clf_train.csv').pipe(utils.detect_id_col)
df_inf = pd.read_csv('./data/clf_test.csv')
df.head()

# 2. Obtain Features

In [ ]:
df_fea, labels = utils.split_feature_target(df, 'Survived')
df_fea.head()

In [ ]:
df_fea.Name[-90:-60]

In [ ]:
df_fea['cabin_class'] = df_fea.Cabin.str.slice(0, 1)
df_fea['Pclass'] = df_fea['Pclass'].astype(str)
df_inf['cabin_class'] = df_inf.Cabin.str.slice(0, 1)
df_inf['Pclass'] = df_inf['Pclass'].astype(str)

In [ ]:
df_fea.drop(['Ticket', 'Name', 'Cabin'], axis=1, inplace=True)

In [ ]:
dp = data_processing.DataProcessor(df_fea, 'titanic', True, 15)

In [ ]:
X, y = dp.data.values, labels.values

In [ ]:
X.shape

In [ ]:
clf, params, est = modelling.train_rf_classifier(X, y, n_iter=30)

In [ ]:
model = est(**params)
model.fit(X, y)

In [ ]:
df_inf['Survived'] = model.predict(dp.transform(df_inf).values)

In [ ]:
df_inf = df_inf.reset_index().rename(columns={'passengerid': 'PassengerId'})

In [ ]:
df_inf[['PassengerId', 'Survived']].to_csv('./data/inf.csv', index=False)

In [ ]:
dp.cat_info